## 1. Cleaning
- ลบแถวที่มี missing values
- แปลงชนิดข้อมูลของคอลัมน์ Price และ Area เป็นตัวเลข
- ลบข้อมูลซ้ำ

In [1]:
import pandas as pd

# โหลดข้อมูลจากไฟล์ Excel
df = pd.read_excel("bangkok condo.xlsx", sheet_name="Bangkok Housing Condo Apartment")

# แสดงข้อมูลเบื้องต้น
print(df.info())
print(df.head())

# ลบแถวที่มีค่าว่าง
df_cleaned = df.dropna()

# แปลงชนิดข้อมูล
df_cleaned['Price (THB)'] = pd.to_numeric(df_cleaned['Price (THB)'], errors='coerce')
df_cleaned['Area (sq. ft.)'] = pd.to_numeric(df_cleaned['Area (sq. ft.)'], errors='coerce')

# ตรวจสอบผลลัพธ์
print(df_cleaned.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 563 entries, 0 to 562
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Property Type   563 non-null    object
 1   Location        563 non-null    object
 2   Area (sq. ft.)  563 non-null    int64 
 3   Bedrooms        563 non-null    int64 
 4   Bathrooms       563 non-null    int64 
 5   Price (THB)     563 non-null    int64 
dtypes: int64(4), object(2)
memory usage: 26.5+ KB
None
  Property Type   Location  Area (sq. ft.)  Bedrooms  Bathrooms  Price (THB)
0         Condo  Sukhumvit             700         1          1      2000000
1         House    Ladprao            1500         3          2      5000000
2     Apartment       Siam             900         2          1      3500000
3         Condo    Sathorn            1200         2          2      4500000
4         House   Ratchada            1800         4          3      8000000
<class 'pandas.core.frame.

## 2. Feature Engineering
- Price_per_sqm: คำนวณจาก Price / Area
- is_luxury: ระบุว่าทรัพย์หรูหรือไม่
- District: แยกชื่อเขตจาก Location

In [2]:
# แปลงพื้นที่จาก sq. ft. → sq. m. (1 sq. ft. = 0.092903 sq. m.)
df_cleaned['Area_sqm'] = (df_cleaned['Area (sq. ft.)'] * 0.092903).round().astype(int)

# ฟีเจอร์: ราคาต่อ ตร.ม.
df_cleaned['Price_per_sqm'] = (df_cleaned['Price (THB)'] / df_cleaned['Area_sqm']).astype(int)

# ฟีเจอร์: ระดับความหรูหรา (Affordable → Ultra Luxury)
def luxury_level(price_per_sqm):
    if price_per_sqm >= 300000:
        return 'Ultra Luxury'
    elif price_per_sqm >= 250000:
        return 'Luxury'
    elif price_per_sqm >= 150000:
        return 'Mid-range'
    else:
        return 'Affordable'

df_cleaned['Luxury_Level'] = df_cleaned['Price_per_sqm'].apply(luxury_level)

# ดูผลลัพธ์เฉพาะฟีเจอร์ที่สำคัญ
df_cleaned[['Area_sqm', 'Price (THB)', 'Price_per_sqm', 'Luxury_Level']].head()


,Area_sqm,Price (THB),Price_per_sqm,Luxury_Level
0,65,2000000,30769,Affordable
1,139,5000000,35971,Affordable
2,84,3500000,41666,Affordable
3,111,4500000,40540,Affordable
4,167,8000000,47904,Affordable


Export ข้อมูลที่ผ่านการทำความสะอาดและเพิ่มฟีเจอร์แล้ว

In [3]:
df_cleaned.to_csv("bangkok condo.csv", index=False)

In [4]:
from google.colab import files
files.download("bangkok condo.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>